## OOP

This shows:
* Abstract method.
    * This leaves a gap that the subclass **must** fill.
* Read-only property pattern.
    * No setter property has been defined, so more immutability.
    * Caveat: true immutability is not possible in Python.


In [295]:
from abc import abstractmethod, ABC
from textwrap import dedent
class Animal(ABC):
    def __init__(self, name):
        self.name = name
        # cleaning the () in the __init__
    
    @property
    @abstractmethod
    def pet(self) -> bool:
        pass
    
    @pet.setter
    def pet(self, value) -> bool:
        print('test')
    
    
    # @final from  Python 3.7 
    def info(self):
        info_string = f"""
        Name: {self.name}
        Pet status: {self.pet}
        """
        print(dedent(info_string))
        

In [296]:
class Dog(Animal):
    def __init__(self, name):
        super().__init__(name)
        
    @property
    def pet(self):
        return True

In [297]:
clifford = Dog("Clifford")

In [298]:
clifford.info()


Name: Clifford
Pet status: True



The `pet` attribute is read-only.

In [299]:
clifford.pet = False

AttributeError: can't set attribute

In [300]:
clifford.info()


Name: Clifford
Pet status: True



## Boolean filtering

* Dictionaries are incredibly versatile, even funtions can be keys.
* We can use this to create a clean matching function, similar to a Java switch.
* This is a lot cleaner than the if else alternative.
* This can be useful if we don't want any short circuiting and multiple matches are possible.

In [424]:
def match(x: int):
    bool_dict = {
        lambda num: num + 1 == 2: "That's numberwang!",
        lambda num: num % 2 == 0: "Even Stevens!",
        lambda num: num % 3 == 0: "Divisible by 6."
    }
    result = [result for test, result in bool_dict.items() if test(x)]
    return result

In [425]:
match(1)

["That's numberwang!"]

In [426]:
match(2)

['Even Stevens!']

In [427]:
match(6)

['Even Stevens!', 'Divisible by 6.']

In [428]:
match(-1)

[]

This implementation is better if we want `CASE WHEN` logic, where only the final match is kept.

In [419]:
def match(x: int):
    bool_dict = {
        lambda num: num + 1 == 2: "That's numberwang!",
        lambda num: num % 2 == 0: "Even Stevens!"
    }
    result = [result for test, result in bool_dict.items() if test(x)]
    result.insert(0, None) # insert None at the front in case of no match
    return result[-1]

In [420]:
match(1)

"That's numberwang!"

In [421]:
match(0)

'Even Stevens!'

In [422]:
match(6)

'Even Stevens!'

In [423]:
match(-1)